In [ ]:
import pandas as pd
import numpy as np

# =============================================================================
# PHASE 1: DATA INGESTION & PORTFOLIO PROFILING
# =============================================================================
# Ingesting the raw loan book for initial quality assessment
df = pd.read_excel('financial_loan_data_excel.xlsx')

print("--- PORTFOLIO INVENTORY ---")
print(f"Total Observation Units (Rows): {df.shape[0]:,}")
print(f"Total Feature set (Columns):   {df.shape[1]}")
print(f"Target Variable:               loan_status")
print(f"Temporal Scope:                2021-01-01 to 2022-01-20")

# =============================================================================
# PHASE 2: DATA QUALITY AUDIT (THE 6 DIMENSIONS FRAMEWORK)
# =============================================================================

# --- Dimension 1: Completeness ---
# Identifying data gaps in critical underwriting fields (e.g., emp_title)
null_count = df['emp_title'].isnull().sum()
null_pct = (null_count / len(df)) * 100
print(f"\n[DIMENSION: COMPLETENESS]")
print(f"Missing Values (emp_title): {null_count:,} records ({null_pct:.2f}%)")

# --- Dimension 2: Validity (Value Range Constraints) ---
# Auditing numerical fields to ensure they reside within logical financial boundaries
print(f"\n[DIMENSION: VALIDITY - RANGE AUDIT]")
audit_cols = ['annual_income', 'dti', 'installment', 'int_rate']
for col in audit_cols:
    print(f"{col.replace('_', ' ').title():<15} | Min: {df[col].min():>10.2f} | Max: {df[col].max():>10.2f}")

# --- Dimension 3: Consistency (Inter-Variable Logic) ---
print(f"\n[DIMENSION: CONSISTENCY - BUSINESS RULE ENFORCEMENT]")

# Check 1: Credit Tiering Alignment
# Logic: Average Interest Rate should scale strictly with Credit Grade (A -> G)
grade_consistency = (df.groupby('grade')['int_rate'].mean()) * 100
print("Mean Interest Rate by Grade (%):")
print(grade_consistency)

# Check 2: Payment Tracking Integrity
# Business Rule: For 'Fully Paid' loans, the total recovered amount must not be less than the principal.
# Any record violating this is a data entry error or an accounting discrepancy.
integrity_violation = (df['loan_status'] == 'Fully Paid') & (df['total_payment'] < df['loan_amount'])
violation_count = df[integrity_violation].shape[0]
violation_pct = (violation_count / len(df)) * 100

print(f"Accounting Inconsistencies Detected: {violation_count} records ({violation_pct:.4f}%)")

# Action: Dropping illogical records to prevent model bias
df = df[~integrity_violation]

# --- Dimension 4 & 5: Uniqueness & Timeliness (Timeline Audit) ---
print(f"\n[DIMENSION: TIMELINESS - CHRONOLOGICAL INTEGRITY]")
print("Violations: Date events occurring prior to Loan Issue Date:")

date_features = df.select_dtypes(include='datetime64').columns.to_list()
if 'issue_date' in date_features:
    date_features.remove('issue_date')

def timeline_audit(df, anchor_col, target_col):
    """Flags records where secondary dates precede the primary loan origination date."""
    violations = df[df[target_col] < df[anchor_col]]
    v_count = violations.shape[0]
    v_pct = (v_count / len(df)) * 100
    print(f"   {target_col:<20}: {v_count:>6,} records ({v_pct:>6.2f}%)")

for col in date_features:
    timeline_audit(df, 'issue_date', col)

# =============================================================================
# PHASE 3: DATA REMEDIATION & CLEANING IMPLEMENTATION
# =============================================================================

# 1. Handling Categorical Sparsity
# Imputing 'Unknown' for missing employment titles to maintain record volume while flagging missingness.
df['emp_title'] = df['emp_title'].fillna('Unknown')

# 2. Applying Minimum Underwriting Thresholds
# Removing micro-loans (< $1,000) that fall outside the portfolio's standard risk scope.
df = df[df['loan_amount'] >= 1000]

# 3. Statistical Outlier Mitigation (Winsorization)
# Capping Annual Income at the 99th percentile to prevent extreme values from distorting the gradient.
income_upper_bound = df['annual_income'].quantile(0.99)
df['annual_income_capped'] = np.where(df['annual_income'] > income_upper_bound, 
                                      income_upper_bound, 
                                      df['annual_income'])

print(f"\n--- DATA REMEDIATION COMPLETE ---")
print(f"Final Cleaned Dataset Shape: {df.shape}")